<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=efb10d51c6151a94c9c727f4e73e3a4052988611ae4e7077f2b4d6b18ea557a2
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-18 15:20:23
-------------------
qualified stocks: 88
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.60 K
Current:  1.45 C
-------------------
Today PnL: -81.50 K (-0.56%)
Current PnL: -20.90 L (-13.69%)
CY Booked + Current PnL: -6.94 L (-4.55%)
-------------------
Total profit:  1.60 L
Total loss:  -22.50 L
-------------------
Total Booked + Current PnL: 20.04 L (15.95%)
Total Booked PnL: 40.94 L (32.59%)
Curr Year Booked PnL: 13.96 L (9.63%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.37 C
Est FTT PnL: 91.73 L (63.28%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.25%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,151.67,85.0,M-SC,0.71,164895.0,-9664.0,9696.0,0.64,-5.54,5.88,0.02,234.0,-1.00,1.14,43.43,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,100.64,49.0,M-SC,3.60,86460.0,-14317.0,14413.0,-0.65,-14.21,16.67,0.09,253.0,-0.99,0.60,13.59,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-14.93,49.0,X-MC,3.67,88920.0,-1758.0,16664.0,0.27,-1.94,18.74,16.44,63.0,-0.11,0.61,17.85,X40,ATH,PHARMA
35,ICICIGI,2252.93,-15.31,56.0,X-MC,7.54,188418.0,14098.0,21103.0,-0.83,8.09,11.20,20.19,68.0,0.67,1.30,24.42,X40,ATH,INSURANCE
84,VOLTAS,1530.00,-2.32,53.0,X-MC,3.67,207615.0,15873.0,21883.0,0.74,8.28,10.54,19.69,78.0,0.73,1.43,15.64,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,112.74,69.0,H-SC,16.31,144768.0,-73728.0,101917.0,10.41,-33.74,70.40,12.90,161.0,-0.72,1.00,20.37,OX40N,NTT,FINANCE
64,SATIN,274.00,-22.47,63.0,H-SC,2.43,238698.0,-37586.0,180790.0,1.97,-13.60,75.74,51.83,146.0,-0.21,1.65,16.28,XY24,NTT,FINANCE
12,BERGEPAINT,680.00,-2.91,73.0,X-MC,2.62,237694.0,10355.0,37698.0,1.52,4.56,15.86,21.14,74.0,0.27,1.64,33.98,XY24,NTT,PAINTS
51,MASFIN,398.61,-16.44,59.0,H-SC,7.17,95145.0,-2835.0,24443.0,1.47,-2.89,25.69,22.05,164.0,-0.12,0.66,38.30,XR,ATH,FINANCE
18,COFFEEDAY,80.00,-51.71,33.0,L-SC,20.71,71021.0,-42528.0,82576.0,1.12,-37.45,116.27,35.27,270.0,-0.52,0.49,73.83,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SURYODAY,240.00,53.51,44.0,H-SC,13.11,140627.0,-38444.0,104176.0,-4.59,-21.47,74.08,36.71,167.0,-0.37,0.97,39.53,XR,NTT,BANKS
81,VAIBHAVGBL,521.00,66.39,52.0,H-SC,10.16,144328.0,-38447.0,149524.0,-3.60,-21.04,103.60,60.77,159.0,-0.26,1.00,29.98,XR,NTT,JEWELLERY
13,BSOFT,831.70,-11.87,53.0,H-SC,8.32,96543.0,-44110.0,115543.0,-3.13,-31.36,119.68,50.79,151.0,-0.38,0.67,12.41,XR,ATH,IT
56,QUESS,986.00,-53.72,31.0,X-SC,37.21,46205.0,-18801.0,169729.0,-3.06,-28.92,367.34,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
26,GILLETTE,11206.78,-19.05,29.0,X-SC,5.95,250170.0,-3726.0,86033.0,-2.88,-1.47,34.39,32.42,85.0,-0.04,1.73,13.41,X40,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.93,46.0,M-SC,10.10,146973.0,-671.0,159995.0,-0.91,-0.45,108.86,107.91,216.0,-0.00,1.01,2.53,OX40N,ATH,CABLES
70,STARHEALTH,761.00,36.05,66.0,H-SC,15.64,280323.0,-800.0,133658.0,-0.95,-0.28,47.68,47.26,171.0,-0.01,1.93,50.19,XY24,NTT,INSURANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.93,46.0,M-SC,10.10,146973.0,-671.0,159995.0,-0.91,-0.45,108.86,107.91,216.0,-0.00,1.01,2.53,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,151.67,85.0,M-SC,0.71,164895.0,-9664.0,9696.0,0.64,-5.54,5.88,0.02,234.0,-1.00,1.14,43.43,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,100.64,49.0,M-SC,3.60,86460.0,-14317.0,14413.0,-0.65,-14.21,16.67,0.09,253.0,-0.99,0.60,13.59,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-65.98,48.0,H-SC,2.24,222696.0,-46971.0,83311.0,-0.10,-17.42,37.41,13.47,158.0,-0.56,1.54,13.23,XY24,NTT,PAINTS
44,JCHAC,2282.00,19986.96,66.0,M-SC,1.45,98207.0,-29598.0,29580.0,0.18,-23.16,30.12,-0.01,233.0,-1.00,0.68,11.20,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,21.85,59.0,M-SC,36.46,164340.0,15120.0,129056.0,-0.58,10.13,78.53,96.62,245.0,0.12,1.13,77.85,XY25,NTT,FINANCE
38,INDIAMART,4810.62,-47.96,49.0,H-SC,4.57,129447.0,6111.0,125512.0,-1.34,4.95,96.96,106.72,139.0,0.05,0.89,29.49,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.96,49.0,H-SC,4.57,129447.0,6111.0,125512.0,-1.34,4.95,96.96,106.72,139.0,0.05,0.89,29.49,AR,ATH,MISC
25,FINCABLES,1641.55,-18.93,46.0,M-SC,10.10,146973.0,-671.0,159995.0,-0.91,-0.45,108.86,107.91,216.0,-0.00,1.01,2.53,OX40N,ATH,CABLES
86,WIPRO,420.00,-15.15,46.0,M-LC,5.59,149315.0,-1630.0,111090.0,-1.32,-1.08,74.40,72.51,99.0,-0.01,1.03,4.88,XR,NTT,IT
85,WHIRLPOOL,2270.00,-43.39,35.0,M-SC,8.43,110952.0,-3846.0,97893.0,-2.40,-3.35,88.23,81.92,236.0,-0.04,0.77,31.35,XR,NTT,DURABLES
87,ZYDUSLIFE,1286.17,-15.85,33.0,H-MC,5.41,205298.0,-4276.0,78937.0,-0.62,-2.04,38.45,35.63,119.0,-0.05,1.42,14.29,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,10.47,24.0,X-SC,10.33,84245.0,-44425.0,89721.0,0.04,-34.53,106.50,35.20,92.0,-0.50,0.58,0.04,X40,NTT,FOOTWEAR
77,TMPV,1065.00,-20.63,25.0,X-LC,0.21,243300.0,-33110.0,454265.0,-0.34,-11.98,186.71,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
34,HONAUT,58357.33,-26.46,28.0,X-SC,1.15,104580.0,-23358.0,70487.0,-1.66,-18.26,67.40,36.84,90.0,-0.33,0.72,7.06,X40N,ATH,ELECTRICAL
23,DMART,5391.45,-18.05,28.0,X-LC,5.24,199830.0,-10577.0,69741.0,-0.98,-5.03,34.90,28.12,19.0,-0.15,1.38,17.41,X40N,ATH,FMCG
26,GILLETTE,11206.78,-19.05,29.0,X-SC,5.95,250170.0,-3726.0,86033.0,-2.88,-1.47,34.39,32.42,85.0,-0.04,1.73,13.41,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TMPV,1065.00,-20.63,25.0,X-LC,0.21,243300.0,-33110.0,454265.0,-0.34,-11.98,186.71,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
43,ITC,452.00,-40.74,45.0,X-LC,0.90,198387.0,-1751.0,22636.0,-0.34,-0.87,11.41,10.44,5.0,-0.08,1.37,4.64,X40,NTT,FMCG
34,HONAUT,58357.33,-26.46,28.0,X-SC,1.15,104580.0,-23358.0,70487.0,-1.66,-18.26,67.40,36.84,90.0,-0.33,0.72,7.06,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-20.57,63.0,X-MC,1.33,183550.0,6812.0,47925.0,-0.66,3.85,26.11,30.97,75.0,0.14,1.27,16.90,X40,ATH,INSURANCE
67,SIEMENS,4671.50,-0.13,64.0,H-LC,1.68,162585.0,-23510.0,70985.0,0.59,-12.63,43.66,25.51,49.0,-0.33,1.12,19.81,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,PAGEIND,51769.93,-29.32,35.0,X-MC,7.93,157160.0,-6580.0,49914.0,-0.46,-4.02,31.76,26.47,55.0,-0.13,1.08,0.00,X40,ATH,APPARELS
56,QUESS,986.00,-53.72,31.0,X-SC,37.21,46205.0,-18801.0,169729.0,-3.06,-28.92,367.34,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
77,TMPV,1065.00,-20.63,25.0,X-LC,0.21,243300.0,-33110.0,454265.0,-0.34,-11.98,186.71,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,10.47,24.0,X-SC,10.33,84245.0,-44425.0,89721.0,0.04,-34.53,106.50,35.20,92.0,-0.50,0.58,0.04,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-19.01,32.0,X-MC,6.73,174791.0,-25715.0,122861.0,-0.75,-12.83,70.29,48.45,64.0,-0.21,1.21,0.79,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,986.00,-53.72,31.0,X-SC,37.21,46205.0,-18801.0,169729.0,-3.06,-28.92,367.34,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-41.92,37.0,X-SC,3.04,79233.0,-65927.0,141859.0,-1.51,-45.42,179.04,52.31,91.0,-0.46,0.55,6.06,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,10.47,24.0,X-SC,10.33,84245.0,-44425.0,89721.0,0.04,-34.53,106.50,35.20,92.0,-0.50,0.58,0.04,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-14.93,49.0,X-MC,3.67,88920.0,-1758.0,16664.0,0.27,-1.94,18.74,16.44,63.0,-0.11,0.61,17.85,X40,ATH,PHARMA
34,HONAUT,58357.33,-26.46,28.0,X-SC,1.15,104580.0,-23358.0,70487.0,-1.66,-18.26,67.40,36.84,90.0,-0.33,0.72,7.06,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-26.86,56.0,X-LC,8.35,289887.0,-56069.0,122767.0,-0.59,-16.21,42.35,19.28,1.0,-0.46,2.00,7.17,X40,ATH,IT
41,INFY,2275.00,-19.96,54.0,X-LC,4.06,321119.0,8231.0,163450.0,0.32,2.63,50.90,54.87,2.0,0.05,2.22,11.13,X40,BTT,IT
77,TMPV,1065.00,-20.63,25.0,X-LC,0.21,243300.0,-33110.0,454265.0,-0.34,-11.98,186.71,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
83,VBL,671.64,-18.42,47.0,X-LC,6.03,292491.0,-23351.0,136008.0,-0.70,-7.39,46.50,35.67,4.0,-0.17,2.02,5.44,X40N,ATH,FMCG
43,ITC,452.00,-40.74,45.0,X-LC,0.90,198387.0,-1751.0,22636.0,-0.34,-0.87,11.41,10.44,5.0,-0.08,1.37,4.64,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,-51.71,33.0,L-SC,20.71,71021.0,-42528.0,82576.0,1.12,-37.45,116.27,35.27,270.0,-0.52,0.49,73.83,XR,NTT,HOTELS
6,ASIANTILES,137.00,7744.44,68.0,L-SC,17.74,86085.0,-7725.0,84346.0,0.39,-8.23,97.98,81.67,271.0,-0.09,0.59,66.71,XR,NTT,CERAMICS
51,MASFIN,398.61,-16.44,59.0,H-SC,7.17,95145.0,-2835.0,24443.0,1.47,-2.89,25.69,22.05,164.0,-0.12,0.66,38.30,XR,ATH,FINANCE
71,SURYODAY,240.00,53.51,44.0,H-SC,13.11,140627.0,-38444.0,104176.0,-4.59,-21.47,74.08,36.71,167.0,-0.37,0.97,39.53,XR,NTT,BANKS
81,VAIBHAVGBL,521.00,66.39,52.0,H-SC,10.16,144328.0,-38447.0,149524.0,-3.60,-21.04,103.60,60.77,159.0,-0.26,1.00,29.98,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,151.67,85.0,M-SC,0.71,164895.0,-9664.0,9696.0,0.64,-5.54,5.88,0.02,234.0,-1.00,1.14,43.43,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.26,0.63,84.0,X-LC,10.16,255895.0,4127.0,48620.0,0.85,1.64,19.00,20.95,31.0,0.08,1.77,37.86,X40,ATH,PAINTS
81,VAIBHAVGBL,521.00,66.39,52.0,H-SC,10.16,144328.0,-38447.0,149524.0,-3.60,-21.04,103.60,60.77,159.0,-0.26,1.00,29.98,XR,NTT,JEWELLERY
13,BSOFT,831.70,-11.87,53.0,H-SC,8.32,96543.0,-44110.0,115543.0,-3.13,-31.36,119.68,50.79,151.0,-0.38,0.67,12.41,XR,ATH,IT
40,INDUSINDBK,1800.00,-696.43,67.0,L-MC,5.94,48296.0,-28910.0,54304.0,-0.79,-37.45,112.44,32.89,259.0,-0.53,0.33,33.00,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.54
1,25,44.34
2,50,75.13


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.77
MC    28.64
LC    24.61
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.99
X40      20.50
X40N     11.56
XY25     10.11
XR        9.49
AR        9.04
OX40N     7.74
X200      1.79
SR        1.00
MH        0.80
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.35
X-MC    22.27
X-LC    19.69
M-SC    13.26
X-SC     7.73
H-MC     4.68
H-LC     2.91
L-SC     1.43
M-MC     1.36
M-LC     1.03
L-LC     0.98
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.56,-4.90,38.82
IT,13.33,-15.44,75.08
FINANCE,10.61,-10.51,62.48
MISC,7.42,-20.84,81.79
PAINTS,6.09,-4.78,20.35
ELECTRICAL,5.65,-10.37,49.50
INSURANCE,4.50,3.08,31.07
PHARMA,3.40,-1.31,35.29
AUTO,3.35,-18.21,131.54


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3385854.0,21
XR,1281310.0,13
AR,1261691.0,10
X40,1000379.0,14
X40N,697393.0,9
OX40N,671162.0,10
XY25,467717.0,7
SR,274474.0,2
MH,67555.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3475861.0,25
M-SC,1429161.0,16
X-LC,1215716.0,11
X-MC,1165861.0,16
X-SC,845162.0,8
H-MC,400386.0,3
L-SC,253791.0,3
H-LC,136010.0,2
M-LC,111090.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1167385.0      6
           AR         850095.0      5
           XR         802717.0      7
M-SC       XY24       758716.0      6
X-LC       XY24       524238.0      2
X-MC       X40        415689.0      7
X-LC       X40        408936.0      5
X-SC       XY24       403357.0      3
X-MC       XY24       347929.0      3
H-SC       OX40N      313635.0      4
           SR         274474.0      2
M-SC       OX40N      270658.0      5
X-SC       X40N       266051.0      3
X-MC       X40N       225593.0      4
H-MC       AR         216157.0      2
X-LC       X40N       205749.0      2
H-MC       XY24       184229.0      1
X-MC       XY25       176650.0      2
X-SC       X40        175754.0      2
L-SC       XR         166922.0      2
M-SC       XR         146277.0      2
           XY25       129056.0      1
           AR         124454.0      2
M-LC       XR         111090.0      1
L-SC       OX40N       86869.0      1
X-LC       XY25        76793.0      2
H-LC       AR          70985.0      1
H-SC       MH          67555.0      1
H-LC       X200        65025.0      1
L-MC       XR          54304.0      1
M-MC       XY25        47842.0      1
L-LC       XY25        37376.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
